<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

BeautifulSoup is used to scrape the data from wikipedia page, and turned to Pandas dataframe.

Import required packages

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

Scrape Table from wikipedia page

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
soup = BeautifulSoup(requests.get(url).content, 'lxml')
table = soup.find_all('table')[0] 
df = pd.DataFrame()

The dataframe will consist of three columns: Postal Code, Borough, and Neighborhood.
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [5]:
header = []
for th in table.find_all("th"):
    # remove new line and spaces from left and right        
    header.append(th.text.replace('\n', ' ').strip())

for tr in table.tbody.find_all("tr"): # find all <tr> from table's tbody
    t_row = {}
    add_row = True
    for td, th in zip(tr.find_all("td"), header): 
        t_row[th] = td.text.replace('\n', '').strip()
        if th == 'Borough' and (t_row[th] == 'Not assigned' or t_row[th] == ''):
            add_row = False
        
    if add_row:
        df = df.append(t_row, ignore_index=True)

In assigment instruction it's said : More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

However, the wikipedia latest update the Postal Code is already unique, and the neighborhoods are combined with / separator, nevertheless the logic to combine is provided, and the / is replaced with a comma.

In [7]:
df = df[header]
df = df.groupby(['Postal code','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df['Neighborhood'] = df['Neighborhood'].str.replace('/',',')
df.rename(columns={'Postal code':'Postal Code'}, inplace='True')
df.head(10)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park"
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge"
8,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff , Cliffside West"


Also said in the assignment instruction : If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
The wikipedia latest update doesn't have any Not assigned neghborhood, nevertheless the code is provided as well.

In [8]:
df.loc[df['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df['Borough']

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [9]:
df.shape

(103, 3)